<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objective" data-toc-modified-id="Objective-1">Objective</a></span></li><li><span><a href="#The-Chain-Rule" data-toc-modified-id="The-Chain-Rule-2">The Chain Rule</a></span></li><li><span><a href="#Activations" data-toc-modified-id="Activations-3">Activations</a></span><ul class="toc-item"><li><span><a href="#ReLU" data-toc-modified-id="ReLU-3.1">ReLU</a></span></li><li><span><a href="#Softmax" data-toc-modified-id="Softmax-3.2">Softmax</a></span></li></ul></li><li><span><a href="#Cross-Entropy-Loss" data-toc-modified-id="Cross-Entropy-Loss-4">Cross Entropy Loss</a></span></li><li><span><a href="#Linear-Layers" data-toc-modified-id="Linear-Layers-5">Linear Layers</a></span></li><li><span><a href="#Putting-It-All-Together" data-toc-modified-id="Putting-It-All-Together-6">Putting It All Together</a></span><ul class="toc-item"><li><span><a href="#Trainer" data-toc-modified-id="Trainer-6.1">Trainer</a></span></li></ul></li><li><span><a href="#Training-Data" data-toc-modified-id="Training-Data-7">Training Data</a></span><ul class="toc-item"><li><span><a href="#Pre-process-Data" data-toc-modified-id="Pre-process-Data-7.1">Pre-process Data</a></span></li><li><span><a href="#Datasets" data-toc-modified-id="Datasets-7.2">Datasets</a></span></li></ul></li><li><span><a href="#Train" data-toc-modified-id="Train-8">Train</a></span></li></ul></div>

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
# Set seed for reproducibility
seed = 9
np.random.seed(seed)

---------

# Objective

The goal of this notebook is to write a simple two-layer feedforward neural net in python using just numpy. It's very much inspired by [this](https://sgugger.github.io/a-simple-neural-net-in-numpy.html#a-simple-neural-net-in-numpy) fantastic blog by Sylvain Gugger. We won't pretend that this notebook will add upon or improve Sylvain's post. We just want to explain things in our own way to help us understand things a little bit more.

----------

# The Chain Rule

Let's start with the thing that really makes networks tick--back propagation. We like to think of it as just the chain rule, since that's really all it is. The goal of back propagation is to nudge the weights of a network in whichever direction the loss is decreasing the quickest. This means we need to know the gradient of the loss function.

The loss function takes the network's final output and our labels and returns a single number. For classification, the outputs are typically the probabilities (or log-probabilities) of each class, so have shape $(n_{obsv}, \, n_{classes})$. Those probabilities are the output of the final activation layer (e.g. log-softmax), which are themselves the output of the final linear layer, which are themselves--just kidding! We think we've got the picture.

The network's final output is just a composition of all the layers of the network--linear and activation--which means computing the gradient of the loss with respect to each set of linear weights will involve lots and lots of chain rule. To help us write things in a nicer way, we'll denote the $i$-th layer by $f_i$, so that a network with depth $d$ can be expressed as the composition of $d$ functions:

$$x \quad \to \quad f_d \circ f_{d-1} \circ \dots \circ f_1(x)$$

Now, although the loss function is not really a layer, it'll make our lives easier if we let $f_d$ be the loss function. It'll also help if we tighten the notation up a bit and let $x_i$ denote the output of the $i$-th layer:

\begin{align}
    x_1 &= f_1(x) \\
    x_2 &= f_2(x_1) = f_2(f_1(x)) \\
    &\vdots \\
    x_d &= f_d(x_{d-1}) = f_d \circ f_{d-1} \circ \dots \circ f_1(x) \\
\end{align}

Now we can express the gradient of the loss with respect to the network's final output $x_{d-1}$ as

$$\frac{\partial \text{loss}}{\partial x_{d-1}} = \frac{\partial f_d}{\partial x_{d-1}}(x_{d-1}) $$

For the gradient with respect to the output $x_{d-2}$ of the network's penultimate layer, we can use the fact that $x_{d-1} = f_{d-1}(x_{d-2})$ to write 

$$\frac{\partial \text{loss}}{\partial x_{d-2}} = \frac{\partial f_d}{\partial x_{d-1}} \times \frac{\partial f_{d-1}}{\partial x_{d-2}}(x_{d-2})$$

Note that computing the gradient with respect to the output of the network's second to last year involves the gradient with respect to the output of the final layer. This means we first have to compute the gradient of the final layer and then pass that gradient backwards through the network to compute the next gradient. This is why it's called "back propagation" and not "just the chain rule". Envisioning the gradients being passed backwards like a baton is helpful when trying to understand how we update a network's weights.

In our discussion so far we just considered a single variable $x$ as the input to our network. The discussion is a bit more complex when considering multiple variables $x_1, \dots, x_m$ of input but the gist is the same (it involves the Jacobian--although we'll pretend we didn't hear that).

-------

# Activations

If we let $f$ be an activation function and $y = f(x)$ its output given the output $x$ of the previous layer, we can express the gradient as

$$\frac{\partial \text{loss}}{\partial f} = \frac{\partial \text{loss}}{\partial y} \times \frac{\partial f}{\partial x} = \frac{\partial \text{loss}}{\partial y} \times f'(x)$$ 

Remember that the gradient $\text{grad}$ of the loss with respect to $y$ will have already been computed and passed backwards when it's time to compute the gradient with respect to $x$. This means that all we need to do is multiply $\text{grad}$ by $f'(x)$ element-wise. Nice, right? Note that we'll need to cache $x$ for computing the backward pass.

## ReLU

All of our activation functions except the last will be ReLUs:

\begin{align}
    \text{ReLU}(x) = \text{max}(x, 0) = \begin{cases} x & \text{if } x > 0 \\ 0 & \text{otherwise} \end{cases}
\end{align}

You might recognize its derivative as the Heavside function,

\begin{align}
    \text{ReLU}'(x) = \begin{cases} 1 & \text{if } x > 0 \\ 0 & \text{otherwise} \end{cases}
\end{align}

In [3]:
class ReLU:
    """Container for computing the forward and backward pass of ReLU."""
    
    def forward(self, x):
        """Pass a mini-batch through ReLU."""
        self.x = x
        return np.where(x > 0, x, 0)
    
    def backward(self, grad):
        """Return the gradient where x is positive, otherwise zero."""
        return np.where(self.x > 0, grad, 0)

## Softmax

Our final activation will be softmax. Given a single observation $x = [x_1, \dots, x_m]$ as input, softmax returns $y = [y_1, \dots, y_m]$, where

$$ y_i = \text{softmax}(x_i) = \frac{e^{x_i}}{e^{x_1} + \dots + e^{x_m}} $$

If you use the quotient rule and treat the partials with respect to $x_i$ and $x_j$ separately, you'll find 

$$\frac{\partial y_i}{\partial x_j} = \begin{cases} y_i(1 - y_i) & j = i \\ -y_i y_j & j \neq i \end{cases}$$

Since each of the $y_k$ depend on all of the $x_j$, when we compute the gradient of the loss with respect to $x_j$ every $y_k$ will contribute. Sylvain walks us through the calculation step-by-step and in the end we get

$$\frac{\partial \text{loss}}{\partial x_j} =  y_j \left(g_j - \sum_{k=1}^m g_k y_k \right)$$

where $g_k$ denotes the gradient of the loss with respect to $y_k$. Note that just like with ReLU, we'll need to cache $y$ for computing the backward pass. 

Before we go any further though, let's do a sanity check on the dimensions involved. The above derivation handles a single observation $x$. If instead we're given a mini-batch of dimension $\text{bs} \times m$, then both $y$ and $g$ also have shape $\text{bs} \times m$. To compute the summation term for all observations in one go, we'll multiply $y$ and $g$ element-wise and then take the sum across the rows (`axis=1`), producing a column vector of shape $\text{bs} \times 1$. Each element of the resulting column vector will be subtracted from corresponding row of $g$ via broadcasting. The final output will have shape $\text{bs} \times m$.


Alrighty. Now it's time to translate the above to code. We'll implement a numerically stable version of softmax that avoids overflow, which you can read more about [here](https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/). Sadly, numpy doesn't have an `unsqueeze()` method like pytorch, so we'll need to call `reshape(-1, 1)` or `[:, None]` to periodically to turn row vectors into column vectors.

In [312]:
class Softmax:
    """Container for computing the forward and backward pass of softmax."""
    
    def forward(self, x):
        """Return softmax predictions on a mini-batch."""
#         x = x - x.max(axis=1, keepdims=True)
        self.y = np.exp(x) / np.exp(x).sum(axis=1, keepdims=True)
        return self.y
        
    def backward(self, grad):
        """Return the gradient given the gradient of the loss w.r.t softmax predictions."""
        return self.y * (grad - (grad * self.y).sum(axis=1, keepdims=True))

In [313]:
# Let's walk through the backward pass step-by-step
X = np.array([[1, 2], [1, 1]])
grad = np.array([[1, -1], [1, 1]])

softmax = Softmax()
softmax.forward(X)  # y

array([[0.26894142, 0.73105858],
       [0.5       , 0.5       ]])

In [314]:
# Multiply San Francisco``grad`` and ``y`` element-wise, then sum across rows
(grad * softmax.y).sum(axis=1).reshape(-1, 1)

array([[-0.46211716],
       [ 1.        ]])

In [315]:
# Subtract from ``grad`` via broadcasting
grad - (grad * softmax.y).sum(axis=1).reshape(-1, 1)

array([[ 1.46211716, -0.53788284],
       [ 0.        ,  0.        ]])

In [316]:
# Last, multiply with ``y`` element-wise
softmax.y * (grad - (grad * softmax.y).sum(axis=1).reshape(-1, 1))

array([[ 0.39322387, -0.39322387],
       [ 0.        ,  0.        ]])

-------

# Cross Entropy Loss

Cross entropy takes as input the softmax probabilities $\hat{y}$ and labels $y$ and returns a single number. Each softmax probability $\hat{y}_k$ contributes $-\ln \hat{y}_k$ to the loss whenever the true label is $y_k$. Since $y$ consists of one-hot encoded labels, only one $y_k$ is equal to one and the rest are zero.

$$\text{CE}(y, \hat{y}) = - \sum_{k=1} y_k \ln \hat{y}_k  $$

Since the loss function is the "last layer", there's no chain rule involved. The gradient is simply

$$\frac{\partial \text{CE}}{\partial y_k} = -\frac{y_k}{\hat{y}_k}$$

If you're like me and worry about division by zero errors, you'll be concerned about the softmax layer returning probabilities close to zero. Fret not. Sylvain says that in practice the probabilities are clipped to $10^{-8}$ (usually). This time we'll need to cache both the labels $y_k$ and probabilities $\hat{y}_k$ for computing the backward pass.

In [317]:
class CrossEntropy:
    """Container for computing the forward and backward pass of cross entropy."""
    
    def forward(self, y, y_hat):
        """Return the cross-entropy given labels and soft predictions."""
        self.y, self.y_hat = y, y_hat.clip(min=1e-8, max=1-1e-8)
        return np.where(y==1, -np.log(self.y_hat), 0).sum(axis=1)
    
    def backward(self):
        """Return the gradient of cross entropy w.r.t soft predictions."""
        return np.where(self.y==1, -1 / self.y_hat, 0)

-------

# Linear Layers

The last part of a network we need to model are the linear layers, which consist of weights and biases:

$$Y = XW + b$$

where $X$ is mini-batch of shape $bs \times n_{inp}$ and the weight matrix $W$ has shape $n_{inp} \times n_{out}$. The bias is a column vector of shape $n_{out} \times 1$ and is added to the product $XW$ via broadcasting. The final output has shape $bs \times n_{out}$.

The output $y_i$ given a single input vector $x = [x_1, \dots, x_{n_{inp}}]$ is just the dot product of the $x$ with the $i$-th column of $W$ plus the $i$-th bias term:

$$y_i = \sum_{k=1}^{n_{inp}} x_k w_{ki} + b_i$$.

The gradients with respect to $w_{ki}$ and $b_i$ are

$$ \frac{\partial \text{loss}}{\partial w_{ki}} = \frac{\partial \text{loss}}{\partial y_i} \times \frac{\partial y_j}{\partial w_{ki}} = x_k \frac{\partial \text{loss}}{\partial y_i} $$

and 

$$ \frac{\partial \text{loss}}{\partial b_i} = \frac{\partial \text{loss}}{\partial y_i} \times \frac{\partial y_i}{\partial b_i} = \frac{\partial \text{loss}}{\partial y_i} $$

Let's make this a little more concrete by considering what happens when we have a batch size of 1. As before, let $\text{grad}$ denote the gradients of the loss with respect to the $y_i$. Suppose $n_{inp}=3$ and $n_{out}=2$ so that

\begin{align}
    x &= [x_1, x_2, x_3] \\
    \text{grad} &= [\text{grad}_1, \text{grad}_2]
\end{align}

Then $W$ has shape $(3, 2)$ and the matrix of gradient updates with respect to the weights looks like

\begin{bmatrix}
    x_1 \text{grad}_1 & x_1 \text{grad}_2 \\
    x_2 \text{grad}_1 & x_2 \text{grad}_2 \\
    x_3 \text{grad}_1 & x_3 \text{grad}_2
\end{bmatrix}

When we have a batch size greater than 1, $X$ has shape $(bs, n_{inp})$ and $\text{grad}$ has shape $(bs, n_{out})$. In order to multiply them to get a matrix of shape $(n_{inp}, n_{out})$ we need to resize $X$ to have shape $(bs, n_{inp}, 1)$ and $\text{grad}$ to have shape $(bs, 1, n_{out})$. This way, ordinary matrix multiplication over the last two dimensions gives the same result as above for the entire mini-batch, yielding a tensor of shape $(bs, n_{inp}, n_{out})$. The last thing is to average the gradients over the batch dimension, producing a final update matrix of size $(n_{inp}, n_{out})$.

The gradients of the loss with respect to the weights and bias tell us how to update them on each backward pass, but we still need to compute the gradient with respect to the inputs $x_k$, which will be passed backwards to the previous layer:

$$\frac{\partial \text{loss}}{\partial x_k} = \sum_{k=1}^{n_{inp}} \frac{\partial \text{loss}}{\partial y_k} \times \frac{\partial y_k}{\partial x_k} = \sum_{k=1}^{n_{inp}} \frac{\partial \text{loss}}{\partial y_k} w_{ki}$$

Sylvain tells us that we can write this as 

$$ \text{new grad} = \text{old grad} \times W^t$$

and we believe him because the shapes work out: $\text{old grad}$ has shape $(bs, n_{out})$ and $W$ has shape $(n_{inp}, n_{out})$, meaning $\text{new grad}$ has shape $(bs, n_{inp})$, just as it should.

In [318]:
class Linear:
    """Container for a linear layer holding weights and biases."""
    
    def __init__(self, n_inp, n_out):
        """Initialise weights and biases."""
        self.W = np.random.uniform(-1 / np.sqrt(n_inp), 1 / np.sqrt(n_inp), (n_inp, n_out))
        self.b = np.zeros(n_out)
        
    def forward(self, x):
        """Pass a mini-batch through the layer."""
        self.x = x
        return x @ self.W + self.b
    
    def backward(self, grad):
        """Backpropagate the gradient to the preceding layer."""
        self.grad_W = (self.x[:,:,None] @ grad[:,None,:]).mean(axis=0)
        self.grad_b = grad.mean(axis=0)
        return grad @ self.W.transpose()

----------

# Putting It All Together

We finally have all the pieces in place to build our own network. Taking after $nn.Sequential$ we'll design it to take an arbitrary collection of layers and an arbitrary loss criterion.

In [319]:
class Sequential:
    """Container for a feedforward neural net."""
    
    def __init__(self, layers, criterion, metric):
        """Initialise layers, loss criterion, and evaluation metric."""
        self.layers = layers
        self.criterion = criterion
        self.metric = metric
        
    def forward(self, x):
        """Pass a mini-batch through the network."""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self):
        """Backpropagate gradients to the start of the network."""
        grad = self.criterion.backward()
        for layer in self.layers[::-1]:
            grad = layer.backward(grad)

In [320]:
class SGD:
    """Container for updating a model's weights via SGD."""
    
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
                  
    def step(self):
        """Update the weights and biases of all linear layers."""
        for layer in self.model.layers:
            if isinstance(layer, Linear):
                layer.W -= self.lr * layer.grad_W
                layer.b -= self.lr * layer.grad_b

## Trainer

For convenience, we'll wrap all of the training functionality we need into a `Trainer` class.

In [321]:
def accuracy(y, y_hat):
    """Compute accuracy of a mini-batch given labels and soft predictions."""
    y_pred = np.argmax(y_hat, axis=1)
    return (y_pred == y).mean()

In [322]:
class Trainer:
    """Container for training a feedforward neural net."""
    
    def __init__(self, model, optimizer, train_dl, val_dl):
        self.model = model
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.val_dl = val_dl
        
    def _train(self):
        """Train for a single epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            self.model.backward()
            self.optimizer.step()
            loss += batch_loss
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for multiple epochs."""
        for epoch in range(n_epochs):
            loss = self._train()
            self.optimizer.step()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y, y_hat).sum()
            batch_metric = self.model.metric(y, y_hat)
            metric += len(y) * batch_metric
            loss += batch_loss
            n += len(y)
        return loss / n, metric / n

-----

# Training Data

## Pre-process Data

In [323]:
# First up is breast cancer
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

((569, 30), (569,))

In [324]:
# Train-test-split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train.shape, X_val.shape

((455, 30), (114, 30))

In [325]:
# Normalise
mu, sigma = X_train.mean(), X_train.std()
X_train = (X_train - mu) / sigma
X_val = (X_val - mu) / sigma

## Datasets

In [326]:
class Dataset:
    """Container for returning inputs and targets."""
    
    def __init__(self, X, y):
        self.X = X
        self.y = y.reshape(-1, 1)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __setitem__(self, idx, val):
        self.X[idx], self.y[idx] = val
                
    def __len__(self):
        return len(self.y)

In [327]:
class DataLoader:
    """Container for returning a mini-batch of inputs and targets."""
    
    def __init__(self, ds, batch_size, shuffle=False):
        self.ds = ds
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def shuffle_data(self):
        """Shuffle inputs and targets."""
        idxs = np.random.permutation(len(self.ds))
        self.ds = Dataset(*self.ds[idxs])
        
    def __iter__(self):
        """Yield a mini-batch of inputs and targets."""
        if self.shuffle: self.shuffle_data()
        n_batches = len(self.ds) // self.batch_size
        for i in range(n_batches):
            yield self.ds[i * self.batch_size: (i + 1) * self.batch_size]

In [328]:
# Load training and validation data
train_ds = Dataset(X_train, y_train)
val_ds = Dataset(X_val, y_val)

batch_size=50
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

-----------

# Train

Let's take this thing for a test drive!

In [329]:
# Input and final output dims
n_inp = X_train.shape[1]
n_classes = len(np.unique(y))

# Initialise model with layers, criterion, and metric
metric = accuracy
criterion = CrossEntropy()
layers = [Linear(n_inp, n_classes), Softmax()]
model = Sequential(layers, criterion, metric)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.1)
trainer = Trainer(model, optimizer, train_dl, val_dl)

In [330]:
trainer.train(20)

epoch=  0 | loss= 0.878 | val_loss= 0.873 | val_metric= 0.568
epoch=  1 | loss= 0.863 | val_loss= 0.873 | val_metric= 0.567
epoch=  2 | loss= 0.860 | val_loss= 0.873 | val_metric= 0.563
epoch=  3 | loss= 0.872 | val_loss= 0.873 | val_metric= 0.541
epoch=  4 | loss= 0.860 | val_loss= 0.873 | val_metric= 0.507
epoch=  5 | loss= 0.869 | val_loss= 0.873 | val_metric= 0.546
epoch=  6 | loss= 0.860 | val_loss= 0.873 | val_metric= 0.546
epoch=  7 | loss= 0.863 | val_loss= 0.873 | val_metric= 0.514
epoch=  8 | loss= 0.863 | val_loss= 0.873 | val_metric= 0.504
epoch=  9 | loss= 0.866 | val_loss= 0.873 | val_metric= 0.544
epoch= 10 | loss= 0.856 | val_loss= 0.873 | val_metric= 0.520
epoch= 11 | loss= 0.863 | val_loss= 0.873 | val_metric= 0.536
epoch= 12 | loss= 0.866 | val_loss= 0.873 | val_metric= 0.555
epoch= 13 | loss= 0.863 | val_loss= 0.873 | val_metric= 0.549
epoch= 14 | loss= 0.860 | val_loss= 0.873 | val_metric= 0.504
epoch= 15 | loss= 0.866 | val_loss= 0.873 | val_metric= 0.524
epoch= 1